In [ ]:
%load_ext autoreload
%autoreload 2
%aimport

In [ ]:
%matplotlib inline

In [ ]:
import bliss
import torch
import matplotlib.pyplot as plt

In [ ]:
# check GPU is configured correctly
device = torch.device("cuda:0")
!echo $CUDA_VISIBLE_DEVICES

# Scratch

In [ ]:
from astropy.table import Table

t = Table.read("../../../../data/sweep-330m005-340p000.fits")

In [ ]:
t["TYPE"][0] == "REX"

In [ ]:
c = t["TYPE"]

In [ ]:
c.data

In [ ]:
r = np.array(t["RA"])
d = np.array(t["DEC"])
mag_r = 22.5 - 2.5 * np.log10(np.array(t["FLUX_R"]))

In [ ]:
keep = (
    (r > 336.5825)
    & (r < 336.7461)
    & (d > -1.044)
    & (d < -0.8189)
    & (mag_r < 23.0)
    & ~np.isnan(mag_r)
)

In [ ]:
cat = t[keep]

In [ ]:
cat["BRICKNAME"]

In [ ]:
import torch

objc_type = t["TYPE"].data.astype(str)
is_galaxy = torch.from_numpy(
    (objc_type == "DEV") | (objc_type == "REX") | (objc_type == "EXP") | (objc_type == "SER")
)

In [ ]:
is_galaxy

# PHOTO 

In [ ]:
from bliss.catalog import PhotoFullCatalog

photo_cat = PhotoFullCatalog.from_file("../../../../data/sdss/", 94, 1, 12, 2)

In [ ]:
mags = photo_cat["mags"].reshape(-1)
keep = mags < 17
ra1, dec1 = photo_cat["ra"].reshape(-1)[keep], photo_cat["dec"].reshape(-1)[keep]
np.sort(ra1)

In [ ]:
photo_cat["mags"].min()

In [ ]:
plocs = photo_cat.plocs - 0.5
plt.figure(figsize=(20, 20))
plt.imshow(frame.image[0, 0], vmin=700, vmax=1000)
plt.scatter(plocs[0, :, 1].numpy(), plocs[0, :, 0].numpy(), marker="x", color="r", s=21)

# COADD

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])

bp = 24
frame = instantiate(cfg.plots.frame, cat_type="coadd")
h_topleft = bp
w_topleft = bp
h_end = ((frame.image.shape[2] - 2 * bp) // 4) * 4 + bp
w_end = ((frame.image.shape[3] - 2 * bp) // 4) * 4 + bp
coadd = frame.get_catalog((h_topleft, h_end), (w_topleft, w_end))
coadd.plocs.shape

In [ ]:
coadd.plocs = coadd.plocs + 24 - 0.5
plt.figure(figsize=(20, 20))
plt.imshow(frame.image[0, 0], vmin=700, vmax=1000)
plt.scatter(coadd.plocs[0, :, 1].numpy(), coadd.plocs[0, :, 0].numpy(), marker="x", color="r", s=21)

In [ ]:
frame.wcs.all_pix2world(, , 0)

# Decals

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])

In [ ]:
# ra_lim = (336.5825, 336.7461)
# dec_lim = (-1.044, -0.8189)

bp = 24
frame = instantiate(
    cfg.plots.frame, cat_type="decals", cat_file="../../../../data/sweep-330m005-340p000.fits"
)
h_topleft = bp
w_topleft = bp
h_end = ((frame.image.shape[2] - 2 * bp) // 4) * 4 + bp
w_end = ((frame.image.shape[3] - 2 * bp) // 4) * 4 + bp
hlim = (h_topleft, h_end)
wlim = (w_topleft, w_end)

decals_cat = frame.get_catalog(hlim, wlim)
decals = decals_cat.apply_mag_bin(-np.inf, 23)
decals.n_sources

In [ ]:
plocs = decals.plocs - 0.5
plt.figure(figsize=(20, 20))
plt.imshow(frame.image[0, 0], vmin=700, vmax=1000)
plt.scatter(plocs[0, :, 1].numpy(), plocs[0, :, 0].numpy(), marker="x", color="r", s=21)

In [ ]:
coad.plocs.shape

In [ ]:
torch.sum(decals["galaxy_bools"])

In [ ]:
torch.sum(decals["star_bools"])

In [ ]:
decals.plocs[0, :, 0].max(), decals.plocs[0, :, 1].max()

In [ ]:
decals.height

In [ ]:
frame.image.shape

## bitwise mask

In [ ]:
from astropy.table import Table

t = Table.read("../../../../data/sweep-330m005-340p000.fits")
t = t[10:1000]
b = np.array(t["MASKBITS"])

In [ ]:
?np.uint

In [ ]:
bin(2048)

In [ ]:
bin(0b1110 & 0b100000101)

In [ ]:
import

# BLISS

In [ ]:
import matplotlib.pyplot as plt
import torch

from bliss.inference import SDSSFrame
from bliss.datasets import sdss
from bliss.inference import reconstruct_scene_at_coordinates
from case_studies.sdss_galaxies.plots.main import load_models


from astropy.table import Table

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])


enc, dec = load_models(cfg, device)
bp = enc.border_padding
torch.cuda.empty_cache()